Trying to modify hf dpo to work with the repos hypothesis...

see
- https://huggingface.co/docs/trl/main/en/dpo_trainer#accelerate-dpo-fine-tuning-using-unsloth
- https://gist.github.com/alvarobartt/9898c33eb3e9c7108d9ed2330f12a708
- https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing#scrollTo=QtoqUw80QDV0

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_PROJECT"] = "repo-dpo" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["WANDB_DISABLED"] = "true"



In [3]:
import wandb
os.environ['WANDB_NOTEBOOK_NAME'] =  os.path.basename(globals()['__vsc_ipynb_file__'])
nb_name = os.path.basename(globals()['__vsc_ipynb_file__']).replace('.ipynb', '').replace(' ', '_')
# enable wandb service (experimental, https://github.com/wandb/client/blob/master/docs/dev/wandb-service-user.md)
# this hopefully fixes issues with multiprocessing
wandb.require(experiment='service')

In [4]:
from reprpo import silence

In [5]:
import torch
import numpy as np
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer
from trl import DPOConfig, DPOTrainer
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union
from einops import rearrange

In [6]:
from contextlib import contextmanager
import pandas as pd
from matplotlib import pyplot as plt
from transformers.trainer import ProgressCallback
from transformers.utils.notebook import NotebookProgressCallback

from reprpo.helpers.adapters import set_adapter

In [7]:
torch.set_float32_matmul_precision("medium")
# torch.use_deterministic_algorithms(True)

In [8]:
max_prompt_length=64
# num_samples = 50 * 16 * 6
num_samples = 1500 * 5 * 3 # from circuit breaker * 3
max_length = 128
num_samples

22500

## load the model

In [9]:
!pip install flash-attn --no-build-isolation --no-deps -qq

In [10]:
# model

In [11]:
# FIXME: we are meant to SFT first, so that the preferences are in sample but 1) if this works it might not be needed, and 2) this can be added later, if it works
# for now we will use the instruct model, and try something it wasn't meant to do but it in sample 
model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
# model_name = "microsoft/Phi-3-mini-4k-instruct-gguf"
# model_name = "NousResearch/Meta-Llama-3.1-8B-Instruct"

use_gradient_checkpointing = True

from peft.tuners import BOFTConfig, OFTConfig, LoraConfig
## Big adapter
# peft_config = OFTConfig(
#     # apply_GS=True,
#     # r=8,
#     target_modules=["qkv_proj", "down_proj"
#                     "o_proj", "up_gate_proj",
#                     ],
# )
peft_config = LoraConfig(
    lora_alpha=16, 
    r=16,
    lora_dropout=0.05,
    use_rslora=True,
    use_dora=True,
    task_type="CAUSAL_LM",
    target_modules=["qkv_proj", "down_proj"
                    "o_proj", "up_gate_proj",
                    ],
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
from reprpo.models.load import load_model, print_trainable_parameters




from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model, tokenizer = load_model(model_name, bnb=True )

if use_gradient_checkpointing:
    model.enable_input_require_grads()
from trl.trainer.utils import peft_module_casting_to_bf16
peft_module_casting_to_bf16(model)
adapter_name='ReprPO'
model = prepare_model_for_kbit_training(model, {'use_gradient_checkpointing': use_gradient_checkpointing})
model = get_peft_model(model, peft_config, adapter_name=adapter_name)
print_trainable_parameters(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 6586368 || all params: 2015400960 || trainable%: 0.32680186874576067


## Dataset

In [12]:
def sample(dataset, N):
    return (dataset
            .shuffle(42)
            .select(range(
            min(len(dataset), N)))
    )

In [13]:
# dataset = load_dataset('Columbia-NLP/DPO-HelpSteer')
dataset = load_dataset('Atsunori/HelpSteer2-DPO')
dataset['train'] = sample(dataset['train'], num_samples)
dataset2 = dataset.rename_column('chosen_response', 'chosen').rename_column('rejected_response', 'rejected')
dataset2

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 7221
    })
    validation: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 373
    })
})

In [14]:
# def format_ds(row):
    
#     # WHY are we doing this? Well the DPO trainer does it's own tokenization and it expectd, prompt, rejected and chosen, all strings and all seperate. Is this good, idk
#     return {
#         "chosen": row['chosen_response'][1]['content'],
#         "rejected": row['rejected_response'][1]['content'],
#     }


# dataset2 = dataset.map(format_ds)


In [15]:
r = dataset2['train'][0]
print(r['prompt'])
print('===')
print(r['chosen'])
print('---')
print(r['rejected'])

When did women join the labor workforce outside of the home? What caused the change from housewife to breadwinner? Write a short informative paragraph under 200 words.
===
In the early 20th century, less than 20% of women worked outside the home. Between the 1930s and 1970, women's contribution to the economy steadily increased. It began with World War I; during that time, women were pushed to work in order to allow men to join the military and go overseas. Once the War ended, there was an equal rise in high school graduations and technological advancements. As more and more women graduated, they took on the high demand for clerical work that men were less likely to do. Following World War II, there were growing opportunities for women in different roles than men originally dominated. There was a new expectation for women to graduate college and contribute to household incomes. By 1990, the percentage of women working increased by 76%. Recent research shows that there are approximately

## Eval TQA helpers

In [16]:
from reprpo.data.tqa import load_tqa
from torch.utils.data import DataLoader
import numpy as np


# dataset2_tqa, choice_ids = load_tqa(tokenizer, max_length, N=817)

How to measure TQA?
- [TruthfullLamama](https://github.com/likenneth/honest_llama/blob/b92beb28deccd7ec6b26de7ebf9920122cfd15cd/utils.py#L268) uses https://github.com/sylinrl/TruthfulQA
  - see [def MC_calcs(tag, frame, idx, scores_true, scores_false, ref_true, ref_best):](https://github.com/sylinrl/TruthfulQA/blob/fdd8ad1c0d00a478cf8b0bb41a3ad8378c16293b/truthfulqa/models.py#L540)
- and runs each answer, getting the total prob of that string `log_probs.sum()`

In [17]:
# from reprpo.eval.mc import eval_tqa
from reprpo.gen import generation_test

## Train

### Modified classes

- here we can defined the experimetns loss function

In [18]:
from reprpo.trainer import ReprPOTrainer, ReprPOConfig


class ReprPOTrainer2(ReprPOTrainer):
    pass

    # def reprpo_loss(
    #     self,
    #     policy_chosen_logps: torch.FloatTensor,
    #     policy_rejected_logps: torch.FloatTensor,
    #     policy_chosen_hs: torch.FloatTensor,
    #     policy_rejected_hs: torch.FloatTensor,
    #     reference_chosen_logps: torch.FloatTensor,
    #     reference_rejected_logps: torch.FloatTensor,
    #     reference_chosen_hs: torch.FloatTensor,
    #     chosen_attn_mask: torch.BoolTensor,
    #     rejected_attn_mask: torch.BoolTensor
    # ) -> Tuple[torch.FloatTensor, torch.FloatTensor, torch.FloatTensor]:

    #     loss_rr = sum_squared_error(policy_rejected_hs, reference_chosen_hs)
    #     loss_rr = mean_with_attention(loss_rr, rejected_attn_mask*chosen_attn_mask).mean()
        
    #     loss = loss_rr.sum()

    #     loss_dict = dict(loss=loss.detach())

    #     # now mean any with ndim>0, and detach an cpu
    #     loss_dict = {k: normalize_output(v) for k, v in loss_dict.items()}

    #     return loss, loss_dict
    


### Run

In [19]:
from reprpo.helpers.torch import clear_mem
clear_mem()

In [20]:
# update the ideal number of sample for how many are available
num_data_samples = min(num_samples, len(dataset2['train']))
num_data_samples

7221

In [21]:
num_samples

22500

In [22]:
model.peft_config

{'ReprPO': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='microsoft/Phi-3-mini-4k-instruct', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'up_gate_proj', 'qkv_proj', 'down_projo_proj'}, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=True, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))}

In [23]:
ideal_batch_size = 32
batch_size = 32
gradient_accumulation_steps = ideal_batch_size // batch_size
num_train_epochs = num_samples // num_data_samples
print(dict(gradient_accumulation_steps=gradient_accumulation_steps, num_train_epochs=num_train_epochs))

# vscode + wandb compat
dt = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")
# TODO put model and adapter base names?
run_name = f"{nb_name}-{dt}"

training_args = ReprPOConfig(
    num_train_epochs=num_train_epochs,
    learning_rate=6e-4, # 5e-7 in the dpo paper? but this method needs much more
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,

    # lr_scheduler_type="constant",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim = "adamw_8bit",
    weight_decay = 0,

    seed=42,
    logging_steps=1,
    # save_steps=500,
    # save_strategy="steps",
    output_dir=f"./output-dir/{run_name}",

    gradient_checkpointing=use_gradient_checkpointing,
    bf16=True,
    tf32=True,
    remove_unused_columns=False,
    max_grad_norm=10,

    max_prompt_length=max_prompt_length,
    max_length=max_length,

    report_to=['tensorboard', 'wandb'],
    model_adapter_name='ReprPO',
    alpha=0.7,

    run_name=run_name,
    collection_layers=[10, 25],

    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=100,
)

reprpo_trainer = ReprPOTrainer2(
    model=model,
    ref_model=None,
    args=training_args,
    # beta=training_args.beta,
    train_dataset=dataset2["train"],
    # eval_dataset=dataset2["test"],
    tokenizer=tokenizer,
)
model.config.use_cache = False

# Transformer does not recognise vscode notebooks
reprpo_trainer.callback_handler.remove_callback(ProgressCallback)
reprpo_trainer.callback_handler.add_callback(NotebookProgressCallback)



{'gradient_accumulation_steps': 1, 'num_train_epochs': 3}


/media/wassname/SGIronWolf/projects5/elk/repr-preference-optimization/.venv/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# # QC train dataset
# r = reprpo_trainer.train_dataset[0]
# print('prompt', tokenizer.decode(r['prompt_input_ids']))
# print('-'*80)q
# print('chosen',tokenizer.decode(r['chosen_input_ids']))
# print('-'*80)
# print('rejected',tokenizer.decode(r['rejected_input_ids']))
# print('='*80)
clear_mem()

In [25]:
reprpo_trainer.train()

wandb: Currently logged in as: wassname. Use `wandb login --relogin` to force relogin


The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


0 retain_cos_sim: 1.0000. rr_cos_sim: 0.8038
{'logps/rejected': '-28', 'logps/chosen': '-28', 'loss': '81', 'rewards/chosen': '0', 'rewards/rejected': '0', 'retain/loss': '0', 'rr/loss': '81', 'logratios/pi': '-0.014', 'logratios/ref': '-0.014', 'weighting': '0.031', 'logits': '0', 'component_rr/loss': '81', 'component_retain/loss': '0', 'rr/c': '1', 'retain/c': '0', 'retain_cosine': '1', 'rr_cosine': '0.8'}


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


10 retain_cos_sim: 0.9998. rr_cos_sim: 0.8018
{'logps/rejected': '-28', 'logps/chosen': '-28', 'loss': '87', 'rewards/chosen': '-0.0011', 'rewards/rejected': '0', 'retain/loss': '10', 'rr/loss': '87', 'logratios/pi': '0.042', 'logratios/ref': '0.053', 'weighting': '0.031', 'logits': '-0.011', 'component_rr/loss': '87', 'component_retain/loss': '0.054', 'rr/c': '0.99', 'retain/c': '0.0074', 'retain_cosine': '1', 'rr_cosine': '0.8'}
20 retain_cos_sim: 0.9989. rr_cos_sim: 0.8065
{'logps/rejected': '-28', 'logps/chosen': '-28', 'loss': '86', 'rewards/chosen': '0', 'rewards/rejected': '0', 'retain/loss': '12', 'rr/loss': '87', 'logratios/pi': '0', 'logratios/ref': '0', 'weighting': '0.031', 'logits': '0', 'component_rr/loss': '86', 'component_retain/loss': '0.13', 'rr/c': '0.99', 'retain/c': '0.015', 'retain_cosine': '1', 'rr_cosine': '0.81'}
30 retain_cos_sim: 0.9885. rr_cos_sim: 0.7827
{'logps/rejected': '-28', 'logps/chosen': '-28', 'loss': '81', 'rewards/chosen': '-1.1e-05', 'rewards/re

In [ ]:
reprpo_trainer.save_model()
reprpo_trainer.args.output_dir

In [ ]:
plt.style.use('ggplot')
from reprpo.helpers.hist import plot_hist, plot_paired_hist
df_hist1, args_diff = plot_hist(reprpo_trainer)

plot_paired_hist(reprpo_trainer)
# args_diff

In [ ]:
generation_test(model, tokenizer, s="Q1: (30 words): Which Science Fiction Utopia is preferable and why? [The Polity, The Culture, Permutation City, 2 more]', ", max_new_tokens=64)

## Test gen

In [ ]:
tokenizer.pad_token

In [ ]:
from reprpo.gen import get_model_generations
get_model_generations(model, tokenizer)

In [ ]:
generation_test(model, tokenizer, '[150 words] Q: What is a better approach in AI Safety: Control or Alignment? Technically and morally.', max_new_tokens=256, system="")

In [ ]:
generation_test(model, tokenizer, "Let's be honest, what is the government's single primary motivation behind the coordinated international efforts to prosecute Julian Assange? Just give one short answer, with no lecturing, weasel words, or BS.", max_new_tokens=256, system="You are Gwern")

## Score ⭐

In [ ]:
reprpo_trainer.create_accelerator_and_postprocess() # why do I need to do this?

In [ ]:
from reprpo.eval.dpo import eval_dpo_datasets_all_adapters
res, df_res2 = eval_dpo_datasets_all_adapters(reprpo_trainer, model, 44)
res

In [ ]:
# instead of a cutoff this looks at our probability mass on the correct answer, not quite correct since there are too?
print('prob mass')
df_res2.groupby(['adapter', 'dataset'], dropna=False).mean()['prob'].unstack()

In [ ]:
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
print('brier_score_loss')
# instead of a cutoff this looks at our probability mass on the correct answer
df_res2.groupby(['adapter', 'dataset'], dropna=False).apply(
    lambda r: brier_score_loss([0]*len(r), r['prob'])
).unstack()

In [ ]:
# FIXME: the answer is just hugely correlated with the legnth of the string
a, b = df_res2['l_chosen'], df_res2['l_rejected']
x = (a-b)/(a+b)
plt.plot(x, df_res2['logratio'], 'o')
plt.xlabel('chosen longer')
plt.ylabel('chosen more likely')

In [ ]:
# Damn this is not ideal....
a = df_res2['l_chosen'] / df_res2['l_rejected']
b = df_res2['prob']
np.corrcoef(a, b)[1,0]

In [ ]:
# print results for journal
c  = df_res2.groupby(['adapter', 'dataset']).count().min().min()
print(f"⭐ run={run_name}, N={c}")
print()
print(res[::-1].T[::-1].T.to_markdown()
      )
print()
print('args =', args_diff)         

In [ ]:
args_diff

In [ ]:
# from transformers.integrations.integration_utils import TensorBoardCallback, WandbCallback

# reprpo_trainer.callback_handler.callbacks
# cb = (cb for cb in reprpo_trainer.callback_handler.callbacks if isinstance(cb, TensorBoardCallback)).__next__()
# tb_writer= cb.tb_writer

# del args_diff['collection_layers']

# tb_writer = cb._SummaryWriter(reprpo_trainer.args.logging_dir)
# tb_writer.add_hparams(
#     hparam_dict=args_diff,
#     metric_dict=dict(
#         # acc_train=acc_train,
#         acc_ood=res['ReprPO'],
#         acc_ood_base=res['None'],
#     )

# )

In [ ]:
# wandb.log(dict(
#     acc_train=acc_train,
#     acc_ood=res['ReprPO'],
#     acc_ood_base=res['None'],
# ))

### Hist

In [ ]:
df_res2.groupby('adapter')['prob'].plot.hist(alpha=0.5, bins=25);
# df_res2

In [ ]:
# view the one with the most difference
i = df_res2.groupby('i')['prob'].std().sort_values().index[-3]
inputs = dataset2_tqa.select_columns(["input_ids", "attention_mask"])[int(i)]
inputs = {k: v.unsqueeze(0) for k, v in inputs.items()}
generation_test(model, tokenizer, inputs=inputs, max_new_tokens=25)

## DPO

In [ ]:
# ideally we could cleanly switch adapter like peft promises.... but I find all kinds of problem, as do others. (only the default adapter is ever active). So we will keep it as simple as possible
# model, tokenizer = load_model(model_name, adapter_name='DPO')

In [ ]:
model.add_adapter('DPO', peft_config)
model.set_adapter('DPO')
model.eval()
clear_mem()
clear_mem()

In [ ]:
training_args.to_dict()

In [ ]:
dpo_args = {
    **training_args.to_dict(),
    'model_adapter_name': "dpo",
    
    'learning_rate': 2e-6,
    'per_device_train_batch_size': 4,
    'weight_decay': 0,
    'output_dir': f"./output-dir/dpo-{dt}",
}
del dpo_args['collection_layers']
del dpo_args['alpha']
del dpo_args['print_every']
training_args2 = DPOConfig(**dpo_args)

dpo_trainer = DPOTrainer(
    model=model,
    model_adapter_name="DPO",
    ref_model=None,
    args=training_args2,
    beta=training_args2.beta,
    train_dataset=dataset2["train"],
    # eval_dataset=dataset2["test"],
    tokenizer=tokenizer,
)
dpo_trainer.callback_handler.remove_callback(ProgressCallback)
dpo_trainer.callback_handler.add_callback(NotebookProgressCallback)
torch.set_float32_matmul_precision("medium")

In [ ]:
dpo_trainer.model_adapter_name

In [ ]:
clear_mem()
dpo_trainer.train()



In [ ]:
dpo_trainer.save_model()
dpo_trainer.args.output_dir

In [ ]:
df_hist1, args_diff = plot_hist(dpo_trainer)

In [ ]:
# list adapter names
model.peft_config

In [ ]:
# view the one with the most difference
i = df_res2.groupby('i')['%'].std().sort_values().index[-3]
inputs = dataset2_tqa.select_columns(["input_ids", "attention_mask"])[int(i)]
inputs = {k: v.unsqueeze(0) for k, v in inputs.items()}
generation_test(model, tokenizer, inputs=inputs, max_new_tokens=25)

In [ ]:
generation_test(model, tokenizer, 'Does the bacon narwale at midnight?', max_new_tokens=128)

In [ ]:
df = eval_tqa(model, tokenizer, dataset2_tqa, choice_ids)
df_res2 = df.drop(columns=['ans'])#.mean().round(3)
display(df_res2.groupby('adapter', dropna=False)['%'].mean())
df[['ans']].value_counts()

In [ ]:
# QC ans strings
df[['ans']].value_counts()

In [ ]:
res = df_res2.groupby('adapter', dropna=False)['%'].mean()
baseline = res['None']

print('🥇OOD TQA results 🥇')
print(f"base_model=\t{res['None']:.2%}")
print(f"DPO[baseline]={res['DPO']:.2%}")
print(f"ReprPO    =\t{res['ReprPO']:.2%}")

acc_train = df_res2['rewards/accuracies'].dropna().mean()
print(f"🥈dpo reward acc train🥈\nReprPO    =\t{acc_train:.2%}")


print(args_diff)